<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/PCA_color_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA color augmentation

PCA color augmentation은 ImageNet Classification with Deep Convolutional
Neural Networks(AlexNet)논문에 제시된 data augmentation 기법으로 이 프로젝트를 통해 PCA color augmentation의 아이디어를 이해하고, 파이썬 코드로 구현해 보고자 한다.

# PCA color augmentation의 대략적인 과정과 아이디어

ImageNet Classification with Deep Convolutional Neural Networks 논문에서 설명하는 PCA color augmentation의 과정은 다음과 같다.

1. 이미지의 R,G,B값에 대한 covariance matrix(공분산 행렬)을 구한다.

2. 위에서 구한 covariance matrix 의 고윳값과 고유벡터를 구한다.

3. 고윳값에 평균이 0이고 표준편차가 0.1인 가우시안분포를 기반으로 생성한 각기 다른 random variable $ \alpha $를 곱해준다.

4. 각 열을 고유벡터로 갖는 행렬 $[\textbf{p}_1, \textbf{p}_2, \textbf{p}_3]$과 그에 상응하는 고유값이 순서대로 나열된(위의 $ \alpha $이 곱해진)벡터 [$\lambda_1 \alpha_1, \lambda_2 \alpha_2, \lambda_3 \alpha_3$]를 dot product한다.

5. 위에서 구한 3차원 벡터를 각 픽셀의 RGB값에 더해준다.

과정을 살펴보면 이미지의 RGB값을 이용해 구한 covarian matrix의 고윳값과 고유벡터를 사용해 이미지의 RGB값에 변화를 준다. 이미지의 RGB값의 분포특성을 반영한 covarian matirx의 **고유벡터와 그에 상응하는 고윳값을 곱함으로써 본래의
RGB 분포특성을 어느정도 유지면서 고윳값에 적당한 상수를 곱하는 방식으로 약간의 무작위한 분포차이를 만들어냄을 알 수 있다.**

# 각 과정에 대한 코드구현과 추가이해

이미지 불러오기

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread('/content/drive/MyDrive/워 해머-스페이스 마린.png', cv2.IMREAD_COLOR)

In [ ]:
cv2_imshow(img)
plt.imshow(img[:,:,::-1])

In [ ]:
print(img.shape)
print(img.dtype)

In [ ]:
# cv2로 불로온 컬러이미지의 색상 채널 순서는 BGR이다.
img_r = img.copy()
img_r[:,:,0] = 0
img_r[:,:,1] = 0
cv2_imshow(img_r)

이미지 RGB값 normalization

이미지의 픽셀값을 normalization하는 이유는 후에 이를 기반으로한 상관계수 행렬을 구하기 위함이다. 기반이 되는 data의 scale에 영향을 받는 covariance는 그 값이 data의 scale에 따라 널뛰므로 data의 상관 관계의 방향을 파악할 수는 있어도 그 값자체에서 의미를 찾기 힘들다. 하지만 이를보완한 상관계수는 normalization 과정을 거친 data를 사용하기 때문에 data의 scale에 영향을 받지 않고 data의 상관관계의 방향은 물론 그 강도까지 파악할 수 있다.

In [ ]:
img_norm = img.copy()

In [ ]:
img_norm = img_norm.reshape(-1, 3)
img_norm.shape

In [ ]:
# 이미지의 각 채널의 평균값을 0으로 변환한다.
img_norm = img_norm - np.mean(img_norm, axis = 0)

In [ ]:
# 이미지의 각 채널의 표준편차를 1로 변환한다.
img_norm = img_norm / np.std(img_norm, axis = 0)

covariance mattrix 와 고윳값 고유벡터 구하기.

normalization 과정을 거친 이미지의 RGB값을 기반으로한 covariance matirx(즉, 상관계수 행렬) form은 다음과 같다.

C = $ \begin{pmatrix} Var(R) & Cov(R, G) & Cov(R, B) \\ Cov(G, R) & Var(G) & Cov(G, B) \\ Cov(B, R) & Cov(B, G) & Var(B) \end{pmatrix}$

그렇다면 covariance matirx의 고윳값과 고유벡터에는 어떤의미가 있을까?

covariance matirx의 form을 다시 살펴보면 가장먼저 symmetrix matrix임을 알 수 있다. symmetric matrix는 대각화 가능하며 고유값으로 real number를 가지므로 위의 수식을 구현하는데 무리가 없다.

또, 임의의 3차원 벡터를 곱해보면 기반이된 이미지의 R, G, B값의 분산과 서로의 공분산 특성을 반영해 벡터를 mapping시킨다는 특징도 알 수 있다. 그 의미는 covariance matrix의 고유값중 가장 큰값에 대응하는 고유벡터가 R, G ,B값의 분산을 가장 크게 갖는 축(주 축)이고, 그 다음 큰 고유값에 해당하는 고유벡터는 두번째로 큰 분산을 갖는 축(두번째 주 축), 세번째 고유값에 해당하는 고유벡터는 세번째 주축이라는 뜻 이다. PCA color augmentation은 이 주축에 적당히 작은 임의의 값을 곱해 다시 더해줌으로써 이미지의 R, G, B값의 전체적인 분산특성(아마도 이미지의 색조?)을 헤치지 않으면서도 분산특성이 다른 이미지를 생성하는것이 목적임을 알 수 있다.

In [ ]:
cov = np.cov(img_norm, rowvar = False) # rowvar가 True일시 row가 변수 column이 관측값으로 설정.
print(cov)

In [ ]:
eig_vals, eig_vecs = np.linalg.eigh(cov) # np.linalg.eigh() 함수는 실수 대칭행렬(또는 복소수 에미리미트 행렬)의 eigen value, eigen vector을 반환함.
                                              # eigen value만 반환하는 eigvalsh(), 정사각행렬에서 작동하는 eig(), eigh)()등도 있음.
print(eig_vals)  # 기본 오름차순으로 정렬됨.
print(eig_vecs)  # 열 벡터임.

In [ ]:
# 내림차순으로 정렬
idx = np.argsort(-eig_vals)
eig_vals = eig_vals[idx]
eig_vecs = eig_vecs[:, idx]

print(eig_vals)
print(eig_vecs)

평균이 0이고 표준편차가 0.1인 가우시안 분포에서 랜더한 값 $\alpha$ 3개를 추출한다.

In [ ]:
alphas = np.random.normal(0, 0.1, 3)
print(alphas)

$[\textbf{p}_1, \textbf{p}_2, \textbf{p}_3] [\lambda_1 \alpha_1, \lambda_2 \alpha_2, \lambda_3 \alpha_3]^T$를 계산해 3차원 벡터를 얻는다. 논문에서는 이 값을 그대로 원본이미지의 R, G, B값에 더해준다고 나와있지만 구현에서는 rescale과정을 통해 픽셀값과의 scale 차이를 줄여서 좀더 유의미한 값으로 만들어준다.

In [ ]:
delta = np.dot(eig_vecs, alphas*eig_vals)
delta = (delta*255.).astype('int8')
print(delta.dtype)
print(delta)

위의 3차원 벡터를 각 픽셀의 RGB값에 더해준후 범위를 0~255로 rescale 해준다.

In [ ]:
arg_img = img.reshape(-1,3)
arg_img = arg_img + delta
arg_img = arg_img.reshape(515, 609, 3)
arg_img = np.clip(arg_img, 0.0, 255.0)

In [ ]:
cv2_imshow(arg_img)

원본 이미지를 입력받아 10개의 PCA color argumetation image를 생성하는 메서드
https://qiita.com/koshian2/items/78de8ccd09dd2998ddfc

In [ ]:
def PCA_color_argumentation(img):

    img_norm = img.reshape(-1,3)
    img_norm = img_norm - np.mean(img_norm, axis = 0)
    img_norm = img_norm / np.std(img_norm, axis = 0)

    cov = np.cov(img_norm, rowvar = False)

    en, ev = np.linalg.eigh(cov)

    alphas = np.random.randn(3) * 0.1

    sv = np.dot(ev, en * alphas)
    sv = (sv * 255.0).astype('int8')

    img_out = img.reshape(-1,3) + sv
    img_out = img_out.reshape(img.shape)
    img_out = np.clip(img_out, 0, 255)

    plt.imshow(img_out[:,:,::-1])


    return

In [ ]:
plt.subplot(2, 1, 1)                # nrows=2, ncols=1, index=1
plt.plot(x1, y1, 'o-')
plt.title('1st Graph')
plt.ylabel('Damped oscillation')

plt.subplot(2, 1, 2)                # nrows=2, ncols=1, index=2
plt.plot(x2, y2, '.-')
plt.title('2nd Graph')
plt.xlabel('time (s)')
plt.ylabel('Undamped')

In [ ]:
PCA_color_argumentation(img)

In [ ]:
a = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0,
                31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0,
                35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]
b = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0,
                500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0,
                700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]

In [ ]:
plt.plot(a, b, 'ro')
plt.plot(a1, b1, 'bo')
plt.plot(a2, b2, 'yo')
plt.show

In [ ]:
an = a - np.mean(a)
an = an / np.std(an)

bn = b - np.mean(b)
bn = bn / np.std(bn)

an

In [ ]:
cov = np.cov(an, bn)
cov

In [ ]:
eval, evec = np.linalg.eigh(cov)

In [ ]:
eval

In [ ]:
evec

In [ ]:
delta = np.dot(evec, eval)

In [ ]:
a1 = a + delta[0]
b1 = b + delta[1]
a1

In [ ]:
a2 = a + delta[0]*0.5
b2 = b + delta[1]*0.5

In [ ]:
alphas = np.random.normal(0, 0.1, 3)
delta = np.dot(p, alphas*lambdas)
delta = (delta*255.).astype('int8')
pca_color_image = np.maximum(np.minimum(original_image + delta, 255), 0).astype('uint8')

In [ ]:
np.cov(a, b)

In [ ]:
np.cov(a1, b1)

In [ ]:
a2